In [1]:
from transformers import AutoTokenizer
from vllm import SamplingParams
import string
from vllm import LLM
from datasets import load_dataset
from tqdm import tqdm
import random
from datetime import datetime
import json
import pandas as pd
import argparse
import os


random.seed(42)

model_name = "hatakeyama-llm-team/tanuki_inst_0515test"
n_batch = 1000


parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('jobid', type=int, help='Job ID')
parser.add_argument('model_name', type=str, help='model name')
args = parser.parse_args()

print("Parsed arguments:")
print(args)

job_id = args.jobid
model_name = args.model_name

print(f"Job ID: {job_id}")
print(f"Model Name: {model_name}")

print(job_id,model_name)

#job_id=1

os.environ["CUDA_VISIBLE_DEVICES"] = f"{job_id}"
n_jobs=8
current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")
out_path = f"data/0520orpo_model/job_{job_id}_{current_time_no_symbols}.jsonl"


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-20 13:59:59,229	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
df=pd.read_parquet("data/0520orpo/code_all_10000000000.parquet")
master_records=df.to_dict(orient="records")

In [3]:
print("init models...")
llm = LLM(model=model_name, trust_remote_code=True)

# %%
tokenizer = AutoTokenizer.from_pretrained(model_name)

init models...
INFO 05-20 14:00:24 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/home/hatakeyama/python/EvalPractice/model/0519mult_c_-home-hatakeyama-python-llm-models-hf-epoch2_step12697_inst_0519mult_c-code_all_10000000000-parquet_lr_5e-5_bf16', speculative_config=None, tokenizer='/home/hatakeyama/python/EvalPractice/model/0519mult_c_-home-hatakeyama-python-llm-models-hf-epoch2_step12697_inst_0519mult_c-code_all_10000000000-parquet_lr_5e-5_bf16', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/home/hatakeyama/python/EvalPractice/model/0519mult_c_

INFO 05-20 14:00:25 utils.py:660] Found nccl from library /home/hatakeyama/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-20 14:00:26 selector.py:27] Using FlashAttention-2 backend.
INFO 05-20 14:00:30 model_runner.py:175] Loading model weights took 13.9932 GB
INFO 05-20 14:00:32 gpu_executor.py:114] # GPU blocks: 28751, # CPU blocks: 2048
INFO 05-20 14:00:34 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-20 14:00:34 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-20 14:00:41 model_runner.py:1017] Graph capturing finished in 8 secs.


In [32]:

answer_template="\n\n### 応答:\n"


# プロンプトテンプレートの準備
random.seed(42)
random.shuffle(master_records)

#jobで分割
job_size=int(len(master_records)/n_jobs)
records=master_records[job_id*job_size:(job_id+1)*job_size]

cnt = 0
for i in tqdm(range(int(len(records)/n_batch))):
    # プロンプトの準備
    sampled_records = records[cnt*n_batch:(cnt+1)*n_batch]

    prompts = []
    for record in sampled_records:
        q=record["text"]
        q=q[:q.rfind(answer_template)+len(answer_template)]
        prompts.append(tokenizer.encode(q)[:-1])

    # 推論の実行
    outputs = llm.generate(
        # prompts,
        prompt_token_ids=prompts,
        sampling_params=SamplingParams(
            temperature=0.1,
            max_tokens=512,
        )
    )
    for i, output in enumerate(outputs):
        sampled_records[i]["model_answer"] = output.outputs[0].text

    with open(out_path, "a") as f:
        for record in sampled_records:
            #record.pop("text")
            f.write(json.dumps(record, ensure_ascii=False))
            f.write("\n")

    cnt += 1

    break

  0%|          | 0/25886 [00:00<?, ?it/s]








  0%|          | 0/25886 [00:06<?, ?it/s]


In [33]:
len(prompts),len(sampled_records),len(outputs)

(10, 10, 10)

In [34]:
outputs

[RequestOutput(request_id=39, prompt=None, prompt_token_ids=[272, 1411, 276, 269, 12311, 273, 680, 530, 297, 3994, 278, 269, 684, 5427, 271, 357, 379, 366, 271, 3754, 298, 270, 381, 2023, 273, 950, 1219, 274, 4074, 354, 1045, 2850, 273, 1530, 3314, 270, 8, 8, 48979, 272, 3994, 327, 8, 4242, 10523, 810, 7259, 6476, 293, 296, 3955, 312, 272, 39181, 7314, 322, 8692, 62115, 312, 355, 272, 34888, 25252, 28124, 380, 10798, 1249, 980, 519, 3955, 312, 272, 62115, 272, 16116, 340, 296, 16373, 312, 355, 1638, 4545, 627, 6679, 1619, 355, 13169, 4206, 287, 21015, 377, 766, 340, 642, 15024, 347, 355, 4160, 627, 11351, 12863, 287, 627, 28793, 369, 272, 5777, 355, 7767, 13291, 12762, 436, 5632, 26633, 347, 296, 3022, 627, 355, 2521, 7735, 8135, 284, 8, 8, 34642, 17365, 419, 340, 50855, 812, 6976, 477, 909, 5826, 429, 272, 43944, 322, 272, 53183, 355, 14912, 7607, 9768, 284, 519, 3842, 312, 1267, 272, 62115, 372, 6601, 62115, 6976, 419, 849, 21282, 284, 2483, 2173, 287, 766, 419, 8773, 340, 6543, 477,